In [10]:
import cupy as cp
import numpy as np
from cupy import random
import scipy as sc
from scipy import linalg
import matplotlib as plt
import numpy as np
from numpy import genfromtxt
from scipy.stats import beta
from scipy.stats import norm
from scipy.stats import gamma
from scipy.stats import invgamma
import matplotlib.pyplot as plt
from sympy import Matrix
from datetime import datetime
import seaborn as sns
import math
import time
ordqz = sc.linalg.ordqz
svd = sc.linalg.svd
import pandas as pd
resh  = cp.reshape
norm = cp.random.standard_normal
zeros = cp.zeros
import scipy as sc
inv =  sc.linalg.inv
resh = np.reshape
m = np.matmul
eig = sc.linalg.eig
det = cp.linalg.det


# Initialize Parameters
psi = 1.6**-1;epsilon = 1.2;alpha = .3;sigma = .1;rho = .9;rho_k = .9;
delta = 2.3**-1;n1 = 1.1**-1;n2 = 1.1**-1
phi_b = 1.5;phi_s= 0.1;theta = .016;phi_x = 0.5;phi_pi = 1.5;phi_t =.2;rh0= .7;h_w = .5;h_r = 1-h_w
p_u = .5;p_r = .5;p_s = .5;p_e = .5;p_a = .5; p_r_star = .5;
sig_u =.2;sig_r = .2;sig_s = .2;sig_e = .2;sig_a = .2;sig_r_star = .1
# Initialize Parameters
psi = 1.6**-1;epsilon = 1.2;alpha = .3;sigma = .015;rho = .9;rho_k = .93;
delta = 2.3**-1;n1 = 1.1**-1;n2 = 1.1**-1
phi_b = 1.5;phi_s= 0.1;theta = .016;
phi_x = 0.5;phi_pi = 1.5;phi_t =.2;
rh0= .7;h_w = .5;h_r = 1-h_w
p_u = .5;p_r = .5;p_s = .5;p_e = .5;p_a = .5;g = 0.03


#### Nuisance parameters ########
r_sh = 0;a_sh = 1;s_sh = 2;u_sh = 3;e_sh = 4;
C_bar = .60;K_bar = .4;Y_bar = 1;c1 = Y_bar/(Y_bar-K_bar);c2 = K_bar/(Y_bar-K_bar)
alpha_tilde = (1-(1-.99*theta)*(1-theta))**-1;gam = alpha*(psi**-1)*(-epsilon);v_1 = alpha_tilde*.99*theta;v_2 = alpha_tilde*((1-.99*theta)/(1-gam))
phi_1 = (1-alpha*(psi**-1));phi_2 = alpha*(psi**-1)*delta*c1;phi_3 = alpha+(psi**-1)*alpha*delta*c2
phi_4 = alpha*(psi**-1)*h_w;Q = (C_bar**delta)/((sigma*K_bar)**(n2))
z1 = (c1**-1)*c2-(c1**-1)*c2*(1-sigma);z2 = 1;z3 = (c1**-1)*(delta**-1)*(1+((1-phi_b)**-1)*phi_b*h_r);z4 = -(c1**-1)*(delta**-1)
z5 = (c1**-1)*(delta**-1)*((1-phi_b)**-1)*(-phi_b+rho-1)*h_r;z6 = (c1**-1)*(delta**-1)*((1-phi_b)**-1)*(phi_b+phi_s)*p_s*h_r
z7 = (c1**-1)*(delta**-1)*((1-phi_b)**-1)*(phi_t)*h_r;z8 = -(c1**-1)*(c2)*(1-2*sigma)*rho_k*(.996+.004*.42);
z9 = -(c1**-1)*(c2)*(1-2*sigma);z10 = (c1**-1)*(c2)*(2*sigma);z11 = -delta**-1


rho =.89 ;rho_k = .9059
x = .99*rho

#print( 'duration gov bond: ', (x*(1-x)**-2)/12 )
xk = .99*rho_k
c1 = .997+.003*.42
#print( 'duration corp bond: ', (xk*(1-xk)**-2)/12*c1 )


# In[3]:


def set_para():
    para=  np.zeros([30])
    para[0] =psi;
    para[1] = epsilon;
    para[2] = alpha;
    para[3] = sigma;
    para[4] = rho;
    para[5] = rho_k;
    para[6] = delta;
    para[7] = n1;
    para[8] = n2;
    para[9]= phi_b;
    para[10] =phi_s;
    para[11] = theta;
    para[12] = phi_x;
    para[13] = phi_pi;
    para[14] = phi_t;
    para[15] = rh0;
    para[16] = h_w;
    para[17] = p_u;
    para[18] = p_r;
    para[19] = p_s;
    para[20] = p_e;
    para[21] = p_a;
    para[22] = p_r_star;
    para[23] = sig_u 
    para[24] = sig_r
    para[25] = sig_s
    para[26] = sig_e
    para[27] = sig_a
    para[28] = sig_r_star
    para[29] = g

    return para


# In[4]:


def Solve_ZLB(para):
    m = np.matmul
    inv = np.linalg.inv

    # Define Variables
    psi = para[0];epsilon = para[1];alpha = para[2];sigma = para[3];rho = para[4];rho_k = para[5];delta = para[6];n1 = para[7];n2 = para[8];phi_b = para[9];phi_s= para[10];theta = para[11];phi_x = para[12];phi_pi = para[13];phi_t =para[14];rh0= para[15];h_w = para[16];h_r = 1-h_w;p_u = para[17];p_r = para[18];p_s = para[19];p_e = para[20];p_a = para[21];p_r_star = para[22];
    p_r_star = para[22];sig_u =para[23];sig_r = para[24];sig_s = para[25];sig_e = para[26];sig_a = para[27];sig_r_star = para[28]
    h_r = 1-h_w;
    r = 0;a = 1;s = 2;u = 3;e = 4;
    r_sh = 0;a_sh = 1;s_sh = 2;u_sh = 3;e_sh = 4;i_sh = 5
    b = 5;k = 6;q_km = 7;b_km = 8;w = 9;tau = 10;b_k = 11;i = 12
    y = 13;pi = 14;q = 15;q_k = 16
    C_bar = .6;K_bar = .4;Y_bar = 1;c1 = Y_bar/(Y_bar-K_bar);c2 = K_bar/(Y_bar-K_bar)
    # Set up nuisance parameters
    alpha_tilde = (1-(1-.99*theta)*(1-theta))**-1;gam = alpha*(psi**-1)*(-epsilon);v_1 = alpha_tilde*.99*theta;v_2 = alpha_tilde*((1-.99*theta)/(1-gam))
    phi_1 = (1-alpha*(psi**-1));phi_2 = alpha*(psi**-1)*delta*c1;phi_3 = alpha+(psi**-1)*alpha*delta*c2
    phi_4 = alpha*(psi**-1)*h_w;Q = (C_bar**delta)/((sigma*K_bar)**(n2))
    # Generate Matrices
    GAM0 = np.zeros([17,17]);GAM1 = np.zeros([17,17]);PSI = np.zeros([17,6])    
    # EQ 0 Goverment Bond Demand
    GAM0[b,b] =1
    GAM1[b,b] =1
    GAM0[b,q] = (1-rho)
    GAM0[b,pi] =1
    GAM0[b,tau] = 1
    GAM0[b,s] = 1
    # EQ 1 Capital Evolution 
    GAM0[k,k] =1
    GAM1[k,k] =1-sigma
    GAM0[k,b_km] =-1*( 1-2*sigma)
    GAM0[k,q_km] =-1*(1-2*sigma)
    GAM1[k,b_km] = -2*sigma
    GAM1[k,q_km] = -2*sigma
    GAM1[k,e] = -2*sigma
    # EQ 2 tech shock
    GAM0[a,a] = 1
    GAM1[a,a] = p_a
    PSI[a,a_sh] = sig_a
    # EQ 3 Seinorage shock
    GAM0[s,s] = 1 
    GAM1[s,s] = p_s
    PSI[s,s_sh] = sig_s
    # EQ 4 Demand shock
    GAM0[r,r] = 1
    GAM0[r,i] = p_r_star
    GAM1[r,r] = p_r 
    PSI[r,r_sh] = sig_r 
    # EQ 5 Cost Push shock
    GAM0[u,u] = 1 
    GAM1[u,u] = p_u 
    PSI[u,u_sh] = sig_u 
    # EQ 7 Investment "Credit Freeze" shock
    GAM0[e,e] = 1 
    GAM1[e,e] = p_e 
    PSI[e,e_sh] = sig_e 
    # EQ 8 coporate bond price lag
    GAM0[q_km,q_km] = 1
    GAM1[q_km,q_k] = 1
    # EQ 9 coporate bond demand lag
    GAM0[b_km,b_km] = 1
    GAM1[b_km,b_k] = 1
    # EQ 10 Money/IS equation
    GAM1[y,y] = c1
    GAM1[y,k] = c2
    GAM0[y,y] = c1*delta**-1
    GAM0[y,k] = -c2*delta**-1
    GAM0[y,pi] = delta**-1
    GAM0[y,tau] = -h_r*delta**-1
    #GAM1[y,i] = delta**-1
    GAM1[y,r] = -delta**-1
    # EQ 11 Phillips Curve
    GAM1[pi,pi] = 1
    GAM0[pi,pi] =v_1
    GAM1[pi,w] = -v_2*phi_1
    GAM1[pi,y] = -v_2*phi_2
    GAM1[pi,k] = v_2*phi_3
    GAM0[pi,tau] = v_2*phi_4
    GAM0[pi,u] =1
    # EQ 12 Gov Bond Pricing
    GAM1[q,q] = n1
    GAM1[q,b] = n1
    GAM1[q,y] = -delta*c1
    GAM1[q,k] = delta*c2
    GAM0[q,y] = .99*delta*c1
    GAM0[q,k] = -.99*delta*c2
    GAM0[q,pi] = .99*delta
    GAM0[q,tau] = h_r
    GAM0[q,q] = -.99*rho
    # EQ 13 Corp Bond Pricing
    GAM1[q_k,q] = 1-n2-Q**-1
    GAM1[q_k,b] = n2
    GAM1[q_k,y] = -delta*c1*(Q**-1)
    GAM1[q_k,k] = delta*c2*(Q**-1)
    GAM0[q_k,y] = .99*delta*c1*.994*(Q**-1)
    GAM0[q_k,k] = -.99*delta*c2*.994*(Q**-1)
    GAM0[q_k,pi] = .99*delta*.994*(Q**-1)
    GAM0[q_k,tau] = h_r*.994*(Q**-1)
    GAM0[q_k,q_k] = -(.99*rho_k*.994-.99*rho_k*.42*.006)*(Q**-1)
    # EQ 14 Monetary Policy
    GAM0[i,i] = 1
    GAM1[i,i] = 1
    PSI[i,i_sh] =sig_r_star
    # EQ 15 Wages
    GAM0[w,w] = 1-alpha*psi
    GAM0[w,q_k] = -2*alpha*psi
    GAM0[w,b_k] = -2*alpha*psi
    GAM1[w,q_k] = -2*alpha*psi
    GAM1[w,b_k] = -2*alpha*psi
    GAM0[w,e] = -2*alpha*psi
    GAM0[w,y] = -(delta-psi)*c1
    GAM0[w,a] = -psi
    GAM0[w,k] = delta*c2 
    GAM0[w,tau] = h_w 
    # EQ 16 Tax LOM
    GAM0[tau,tau] =1    
    GAM0[tau,b] = -phi_b
    GAM0[tau,q] = -phi_b 
    GAM0[tau,s] = phi_s 
    GAM1[tau,tau] = phi_t 

    # EQ 17 Corporate Bond Demand
    GAM0[b_k,b_k] = 1
    GAM0[b_k,w] = -1
    GAM1[b_k,q_k] = 1
    GAM1[b_k,b_k] = 1
    GAM0[b_k,y] = -(1-alpha)**-1
    GAM0[b_k,a] = (1-alpha)**-1
    GAM0[b_k,k] = (1-alpha)**-1
    GAM0[b_k,e] =  1
    GAM0[b_k,q_k] = 1

    n_s = 13;n_j = 4
    VV = np.zeros(17)
    F = m(inv(GAM0),GAM1);e_vals, e_vecs = eig(F)
    for i in range(17):
        VV[i] =  ((np.real(e_vals[i])**2)+(np.imag(e_vals[i])**2))**.5
    idx = np.argsort(VV)
    H = e_vecs;H = inv(e_vecs[:,idx]);G_tilde = m(H,PSI)
    # Partion Matrices (Imaginary Component)
    H11 = H[0:n_s,0:n_s];H12 = H[0:n_s,n_s:];H21 = H[n_s:,0:n_s];H22 = H[n_s:,n_s:]
    F11 = np.real(F[0:n_s,0:n_s]);F22 = np.real(F[0:n_s,n_s:]);
    L1 = np.diagflat(e_vals[idx][0:n_s]);L2 = np.diagflat(e_vals[idx][n_s:])
    G1 = PSI[0:n_s,:];G2 = PSI[n_s:,:];G1_tilde= G_tilde[0:n_s,:];G2_tilde= G_tilde[n_s:,:]
    try:
        Q = inv(H22)
    except:
        H22+=1e-2
        Q = inv(H22)

    # Coef for Jumps
    B = np.real(m(m(-inv(Q),inv(L2)),G2_tilde));A = np.real(m(-inv(Q),H21))
    # Coef for States
    C = np.real(F11 -m(F22,m(Q,H21)));D = np.real(m(F22,m(m(-Q,inv(L2)),G2_tilde))+G1)
    # Y_t = A*X_t + B*eps_t 
    # X_t = C*X_t-1 + D*eps_t    
    #print(np.count_nonzero(abs(VV)>=1));print('sien on output: ', A[0,s]>0);print('sien on inflation: ', A[1,s]>0);print('sien on gov bond price: ', A[2,s]>0);print('sien on private bond price: ', A[3,s]>0)
    
    A1 = A[:,5:];A2 = A[:,0:5];C1 = C[5:,5:];C2 = C[5:,0:5]
    A1[:,7] = 0;C1[5:,7] = 0 

    R = np.zeros([5,5])
    R[0,0] =p_r
    R[1,1] =p_a
    R[2,2] =p_s
    R[3,3] =p_u
    R[4,4] =p_e

    # Learning Block
    A11 = A[:,0:5]
    A12 = A[:,5:]
    C11 = C[5:,0:5]
    C12 = C[5:,5:]
    

    
    return R,A1,A2,C1,C2,PSI[:,:-1],A11,A12,C11,C12


# In[5]:


def Solve(para):
    m = np.matmul
    inv = np.linalg.inv

    # Define Variables
    psi = para[0];epsilon = para[1];alpha = para[2];sigma = para[3];rho = para[4];rho_k = para[5];delta = para[6];n1 = para[7];n2 = para[8];phi_b = para[9];phi_s= para[10];theta = para[11];phi_x = para[12];phi_pi = para[13];phi_t =para[14];rh0= para[15];h_w = para[16];h_r = 1-h_w;p_u = para[17];p_r = para[18];p_s = para[19];p_e = para[20];p_a = para[21];
    p_r_star = para[22];sig_u =para[23];sig_r = para[24];sig_s = para[25];sig_e = para[26];sig_a = para[27];sig_r_star = para[28]
    r = 0;a = 1;s = 2;u = 3;e = 4;
    r_sh = 0;a_sh = 1;s_sh = 2;u_sh = 3;e_sh = 4;
    b = 5;k = 6;q_km = 7;b_km = 8;w = 9;tau = 10;b_k = 11;i = 12
    y = 13;pi = 14;q = 15;q_k = 16
    C_bar = .6;K_bar = .4;Y_bar = 1;c1 = Y_bar/(Y_bar-K_bar);c2 = K_bar/(Y_bar-K_bar)
    # Set up nuisance parameters
    alpha_tilde = (1-(1-.99*theta)*(1-theta))**-1;gam = alpha*(psi**-1)*(-epsilon);v_1 = alpha_tilde*.99*theta;v_2 = alpha_tilde*((1-.99*theta)/(1-gam))
    phi_1 = (1-alpha*(psi**-1));phi_2 = alpha*(psi**-1)*delta*c1;phi_3 = alpha+(psi**-1)*alpha*delta*c2
    phi_4 = alpha*(psi**-1)*h_w;Q = (C_bar**delta)/((sigma*K_bar)**(n2))
    # Generate Matrices
    GAM0 = np.zeros([17,17]);GAM1 = np.zeros([17,17]);PSI = np.zeros([17,5])    
    # EQ 0 Goverment Bond Demand
    GAM0[b,b] =1
    GAM1[b,b] =1
    GAM0[b,q] = (1-rho)
    GAM0[b,pi] =1
    GAM0[b,tau] = 1
    GAM0[b,s] = 1
    # EQ 1 Capital Evolution 
    GAM0[k,k] =1
    GAM1[k,k] =1-sigma
    GAM0[k,b_km] =-1*( 1-2*sigma)
    GAM0[k,q_km] =-1*(1-2*sigma)
    GAM1[k,b_km] = -2*sigma
    GAM1[k,q_km] = -2*sigma
    GAM1[k,e] = -2*sigma
    # EQ 2 tech shock
    GAM0[a,a] = 1
    GAM1[a,a] = p_a
    PSI[a,a_sh] = sig_a
    # EQ 3 Seinorage shock
    GAM0[s,s] = 1 
    GAM1[s,s] = p_s
    PSI[s,s_sh] = sig_s
    # EQ 4 Demand shock
    GAM0[r,r] = 1 
    GAM1[r,r] = p_r 
    PSI[r,r_sh] = sig_r 
    # EQ 5 Cost Push shock
    GAM0[u,u] = 1 
    GAM1[u,u] = p_u 
    PSI[u,u_sh] = sig_u 
    # EQ 7 Investment "Credit Freeze" shock
    GAM0[e,e] = 1 
    GAM1[e,e] = p_e
    PSI[e,e_sh] = sig_e
    # EQ 8 coporate bond price lag
    GAM0[q_km,q_km] = 1
    GAM1[q_km,q_k] = 1
    # EQ 9 coporate bond demand lag
    GAM0[b_km,b_km] = 1
    GAM1[b_km,b_k] = 1
    # EQ 10 Money/IS equation
    GAM1[y,y] = c1
    GAM1[y,k] = c2
    GAM0[y,y] = c1*delta**-1
    GAM0[y,k] = -c2*delta**-1
    GAM0[y,pi] = delta**-1
    GAM0[y,tau] = -h_r*delta**-1
    GAM1[y,i] = delta**-1
    GAM1[y,r] = -delta**-1
    # EQ 11 Phillips Curve
    GAM1[pi,pi] = 1
    GAM0[pi,pi] =v_1
    GAM1[pi,w] = -v_2*phi_1
    GAM1[pi,y] = -v_2*phi_2
    GAM1[pi,k] = v_2*phi_3
    GAM0[pi,tau] = v_2*phi_4
    GAM0[pi,u] =1
    # EQ 12 Gov Bond Pricing
    GAM1[q,q] = n1
    GAM1[q,b] = n1
    GAM1[q,y] = -delta*c1
    GAM1[q,k] = delta*c2
    GAM0[q,y] = .99*delta*c1
    GAM0[q,k] = -.99*delta*c2
    GAM0[q,pi] = .99*delta
    GAM0[q,tau] = h_r
    GAM0[q,q] = -.99*rho
    # EQ 13 Corp Bond Pricing
    GAM1[q_k,q] = 1-n2-Q**-1
    GAM1[q_k,b] = n2
    GAM1[q_k,y] = -delta*c1*(Q**-1)
    GAM1[q_k,k] = delta*c2*(Q**-1)
    GAM0[q_k,y] = .99*delta*c1*.994*(Q**-1)
    GAM0[q_k,k] = -.99*delta*c2*.994*(Q**-1)
    GAM0[q_k,pi] = .99*delta*.994*(Q**-1)
    GAM0[q_k,tau] = h_r*.994*(Q**-1)
    GAM0[q_k,q_k] = -(.99*rho_k*.994-.99*rho_k*.42*.006)*(Q**-1)
    # EQ 14 Monetary Policy
    GAM0[i,i] =1
    GAM0[i,y] =-phi_x
    GAM0[i,pi] =-phi_pi
    GAM1[i,i] =rh0
    # EQ 15 Wages
    GAM0[w,w] = 1-alpha*psi
    GAM0[w,q_k] = -2*alpha*psi
    GAM0[w,b_k] = -2*alpha*psi
    GAM1[w,q_k] = -2*alpha*psi
    GAM1[w,b_k] = -2*alpha*psi
    GAM0[w,e] = -2*alpha*psi
    GAM0[w,y] = -(delta-psi)*c1
    GAM0[w,a] = -psi
    GAM0[w,k] = delta*c2 
    GAM0[w,tau] = h_w 
    # EQ 16 Tax LOM
    GAM0[tau,tau] =1    
    GAM0[tau,b] = -phi_b
    GAM0[tau,q] = -phi_b 
    GAM0[tau,s] = phi_s 
    GAM1[tau,tau] = phi_t 

    # EQ 17 Corporate Bond Demand
    GAM0[b_k,b_k] = 1
    GAM0[b_k,w] = -1
    GAM1[b_k,q_k] = 1
    GAM1[b_k,b_k] = 1
    GAM0[b_k,y] = -(1-alpha)**-1
    GAM0[b_k,a] = (1-alpha)**-1
    GAM0[b_k,k] = (1-alpha)**-1
    GAM0[b_k,e] =  1
    GAM0[b_k,q_k] = 1

    n_s = 13
    n_j = 4
    VV = np.zeros(17)
    F = m(inv(GAM0),GAM1)
    e_vals, e_vecs = eig(F)
    for i in range(17):
        VV[i] =  ((np.real(e_vals[i])**2)+(np.imag(e_vals[i])**2))**.5
    idx = np.argsort(VV)
    e_vals, e_vecs = eig(F)

    H = e_vecs
    H = inv(e_vecs[:,idx])
    G_tilde = m(H,PSI)
    # Partion Matrices (Imaginary Component)
    H11 = H[0:n_s,0:n_s]
    H12 = H[0:n_s,n_s:]
    H21 = H[n_s:,0:n_s]
    H22 = H[n_s:,n_s:]
    F11 = np.real(F[0:n_s,0:n_s])
    F22 = np.real(F[0:n_s,n_s:])
    L1 = np.diagflat(e_vals[idx][0:n_s])
    L2 = np.diagflat(e_vals[idx][n_s:])
    G1 = PSI[0:n_s,:]
    G2 = PSI[n_s:,:]
    G1_tilde= G_tilde[0:n_s,:]
    G2_tilde= G_tilde[n_s:,:]
    #Q = inv(H22+1e-30*np.random.standard_normal([n_j,n_j]))
    Q = inv(H22)
    # Coef for Jumps
    B = np.real(m(m(-inv(Q),inv(L2)),G2_tilde))
    A = -np.real(m(inv(Q),H21))
    # Coef for States
    C = np.real(F11 -m(F22,m(Q,H21)))
    D = np.real(m(F22,m(m(-Q,inv(L2)),G2_tilde))+G1)
    # Y_t = A*X_t + B*eps_t 
    # X_t = C*X_t-1 + D*eps_t    
    #print(np.count_nonzero(abs(VV)>=1))
    #print('sien on output: ', A[0,s]>0)
    #print('sien on inflation: ', A[1,s]>0)
    #print('sien on gov bond price: ', A[2,s]>0)
    #print('sien on private bond price: ', A[3,s]>0)
    
    A1 = A[:,5:]
    A2 = A[:,0:5]
    C1 = C[5:,5:]
    C2 = C[5:,0:5]
    
    

    R = np.zeros([5,5])
    R[0,0] =p_r
    R[1,1] =p_a
    R[2,2] =p_s
    R[3,3] =p_u
    R[4,4] =p_e

    A11 = A[:,0:5]
    A12 = A[:,5:]
    C11 = C[5:,0:5]
    C12 = C[5:,5:]
    
    
    
    return R,A1,A2,C1,C2,PSI,A11,A12,C11,C12


# In[6]:


def Set_M(para,PSI,aa):
    m = np.matmul
    inv = np.linalg.inv
    # Set Variables
    y = 0;pi = 1;q = 2;q_k = 3;
    b = 0;k = 1;q_km = 2;b_km = 3;w = 4;tau = 5;b_k = 6;i = 7;
    r = 0;a = 1;s = 2;u = 3;e = 4;
    # Initialize Parameters
    psi = para[0];epsilon = para[1];alpha = para[2];sigma = para[3];rho = para[4];rho_k = para[5];delta = para[6];n1 = para[7];n2 = para[8];phi_b = para[9];phi_s= para[10];theta = para[11];phi_x = para[12];phi_pi = para[13];phi_t =para[14];rh0= para[15];h_w = para[16];h_r = 1-h_w;p_u = para[17];p_r = para[18];p_s = para[19];p_e = para[20];p_a = para[21];
    # Set up nuisance parameters
    C_bar = .4;K_bar = .24;Y_bar = .4+.24;c1 = Y_bar/(Y_bar-K_bar);c2 = K_bar/(Y_bar-K_bar)
    alpha_tilde = (1-(1-.99*theta)*(1-theta))**-1;gam = alpha*(psi**-1)*(-epsilon);v_1 = alpha_tilde*.99*theta;v_2 = alpha_tilde*((1-.99*theta)/(1-gam))
    phi_1 = (1-alpha*(psi**-1));phi_2 = alpha*(psi**-1)*delta*c1;phi_3 = alpha+(psi**-1)*alpha*delta*c2
    phi_4 = alpha*(psi**-1)*h_w;Q = (C_bar**delta)/((sigma*K_bar)**(n2))
    z1 = (c1**-1)*c2-(c1**-1)*c2*(1-sigma);z2 = 1;z3 = (c1**-1)*(delta**-1)*(1+((1-phi_b)**-1)*phi_b*h_r);z4 = -(c1**-1)*(delta**-1)
    z5 = (c1**-1)*(delta**-1)*((1-phi_b)**-1)*(-phi_b+rho-1)*h_r;z6 = (c1**-1)*(delta**-1)*((1-phi_b)**-1)*(phi_b+phi_s)*p_s*h_r
    z7 = (c1**-1)*(delta**-1)*((1-phi_b)**-1)*(phi_t)*h_r;z8 = -(c1**-1)*(c2)*(1-2*sigma)*rho_k*(.996+.004*.42);
    z9 = -(c1**-1)*(c2)*(1-2*sigma);z10 = (c1**-1)*(c2)*(2*sigma);z11 = -delta**-1
    # Generate Matrices
    B10 = np.zeros([4,4]);B11 = np.zeros([4,4]);B12 = np.zeros([4,8]);B13 = np.zeros([4,5]);B14 = np.zeros([4,8]);B20 = np.zeros([8,8]);B21 = np.zeros([8,4]);B22 = np.zeros([8,8]);B23 = np.zeros([8,5])
    #jumps
    # Y
    B10[y,y] += c1;B12[y,k] += c2;B11[y,y] += c1
    B14[y,k] += -c2;B11[y,pi] += delta**-1;B14[y,tau] += -h_r*(delta**-1)
    B12[y,i] += -delta**-1;B13[y,r] += delta**-1
    # Pi
    B10[pi,pi] +=1
    B11[pi,pi] += v_1
    B12[pi,w] += v_2*phi_1
    B10[pi,y] += -v_2*phi_2
    B12[pi,k] += -v_2*phi_3
    B14[pi,tau] += v_2*phi_4
    # q
    B10[q,q] += n2
    B10[q,b] += n2
    B10[q,y] += -delta*c1
    B12[q,k] += -delta*c2
    B11[q,y] += .99*delta*c1
    B14[q,k] += -.99*delta*c2
    B11[q,pi] += .99*delta
    B14[q,tau] += h_r
    B11[q,q] += -rho*.99
    # q_k

    B10[q_k,q_k] += (1-n2-(Q**-1))
    
    B12[q_k,b_k] += n2
    B10[q_k,y] += delta*c1*(Q**-1)
    B12[q_k,k] +=delta*c2*(Q**-1)
    
    B11[q_k,y] += .99*delta*c1*(.994+.42*.006)*(Q**-1)
    B14[q_k,k] += -.99*delta*c2*(.994+.42*.006)*(Q**-1)
    B11[q_k,pi] += .99*delta*(.994+.42*.006)*(Q**-1)
    B14[q_k,tau] += .99*h_r*(.994+.42*.006)*(Q**-1)
    B11[q_k,q_k] += -(.99*rho_k)*(.994+.006*.42)*(Q**-1)
    # states
    #k

    B20[k,k] +=1
    B22[k,k] += 1-sigma
    B21[k,q_k] += (1-2*sigma)*rho_k*.996+.004*.42*(1-2*sigma)*rho_k  
    B20[k,b_k] += -(1-2*sigma) 
    B20[k,b_km] += -2*sigma
    B22[k,q_km] += -2*sigma
    B23[k,e] += -2*sigma
    #b

    B20[b,b] +=1
    B22[b,b] +=1
    B21[b,q] += (rho-1)*rho

    #B21[b,pi] =-1
    B21[b,pi] += -1*v_1
    B20[b,w] += v_2*phi_1
    #B20[b,y] = v_2*phi_2
    B20[b,k] += -v_2*phi_3
    #B23[b,tau] = -v_2*phi_4
    B21[b,q] += ((1-phi_b)**-1)*(phi_b+rho-1)*(-v_2*phi_4-1)
    B21[b,pi] += ((1-phi_b)**-1)*(-phi_b)*(-v_2*phi_4-1)
    B22[b,s] += ((1-phi_b)**-1)*(-phi_s-phi_b)*p_s*(-v_2*phi_4-1)
    B20[b,tau] += -((1-phi_b)**-1)*(-phi_t)*(-v_2*phi_4-1)
    # this went to above
    #B20[b,tau] += 1
    B20[b,s] += 1

    # q_km
    B20[q_km,q_km] += 1
    B21[q_km,q_k] += rho_k*rho_k
    # b_km
    B20[b_km,b_km] += 1
    B22[b_km,b_k] += 1
    #w

    B20[w,w] += 1-alpha*psi
    B20[w,k] += c2*delta

    #B20[w,tau] = h_w
    B21[w,q] += ((1-phi_b)**-1)*(phi_b+rho-1)*(-h_w)
    B21[w,pi] += ((1-phi_b)**-1)*(-phi_b)*(-h_w)
    B22[w,s] += ((1-phi_b)**-1)*(-phi_s-phi_b)*p_s*(-h_w)
    B20[w,tau] += ((1-phi_b)**-1)*(-phi_t)*(-h_w)

    B21[w,q_k] += 2*alpha*psi*(rho_k*.996+.004*.42*rho_k)
    B20[w,b_k] += -2*alpha*psi
    B22[w,b_k] += -2*alpha*psi
    B20[w,q_km] += 2*alpha*psi
    B23[w,e] += 2*alpha*psi
    B23[w,a] += psi
    #B20[w,y] = -(delta*c1-psi)


    #tau

    B20[tau,tau] +=1    
    B20[tau,b] += -phi_b
    B21[tau,q] += phi_b*rho
    B22[tau,tau] +=phi_t    
    B20[tau,s] +=phi_s 
    #b_k

    B20[b_k,b_k] += 1
    B20[b_k,w] += -1
    B20[b_k,q_km] += -1
    B22[b_k,b_k] += 1
    #B20[b_k,y] = -(1-alpha)**-1

    B23[b_k,a] += -(1-alpha)**-1
    B20[b_k,k] += (1-alpha)**-1
    B23[b_k,e] +=  -1
    B21[b_k,q_k] += -1*(rho_k*.996+rho_k*.004*.42)
    # i 

    B20[i,i] += 1
    #B20[i,pi] = -phi_pi
    B21[i,pi] += v_1*phi_pi
    B20[i,w] += -v_2*phi_1*phi_pi
    #B20[i,y] = -v_2*phi_2*phi_pi
    B20[i,k] += v_2*phi_3*phi_pi
    #B20[i,tau] = -v_2*phi_4*phi_pi
    B21[i,q] += ((1-phi_b)**-1)*(phi_b+rho-1)*(v_2*phi_4*phi_pi)
    B21[i,pi] += ((1-phi_b)**-1)*(-phi_b)*(v_2*phi_4*phi_pi)
    B22[i,s] += ((1-phi_b)**-1)*(-phi_s-phi_b)*p_s*(v_2*phi_4*phi_pi)
    B20[i,tau] += -((1-phi_b)**-1)*(-phi_t)*(v_2*phi_4*phi_pi)
    #B20[i,y] = -phi_x
    B22[i,i] +=rh0

    # Expression for Y in terms of States
    #B20[b_k,y] = -(1-alpha)**-1
    B20[b_k,k] += -z1*(1-alpha)**-1
    B21[b_k,y] += z2*(1-alpha)**-1
    B21[b_k,pi] += z3*(1-alpha)**-1
    B20[b_k,i] += -z4*(1-alpha)**-1
    B21[b_k,q] += z5*(1-alpha)**-1
    B20[b_k,s] += -z6*(1-alpha)**-1
    B20[b_k,tau] +=-z7*(1-alpha)**-1
    B21[b_k,q_k] += z8*(1-alpha)**-1
    B20[b_k,b_k] += -z9*(1-alpha)**-1
    B23[b_k,e] += z10*(1-alpha)**-1
    B23[b_k,r] += z11*(1-alpha)**-1
    #B20[w,y] = -(delta*c1-psi)
    B20[b_k,k] += -z1*(delta*c1-psi)
    B21[b_k,y] += z2*(delta*c1-psi)
    B21[b_k,pi] += z3*(delta*c1-psi)
    B20[b_k,i] += -z4*(delta*c1-psi)
    B21[b_k,q] += z5*(delta*c1-psi)
    B20[b_k,s] += -z6*(delta*c1-psi)
    B20[b_k,tau] +=-z7*(delta*c1-psi)
    B21[b_k,q_k] += z8*(delta*c1-psi)
    B20[b_k,b_k] += -z9*(delta*c1-psi)
    B23[b_k,e] += z10*(delta*c1-psi)
    B23[b_k,r] += z11*(delta*c1-psi)
    #B20[i,y] =-1*(phi_x+v_2*phi_4*phi_pi)
    B20[b_k,k] += -z1*(phi_x+v_2*phi_4*phi_pi)
    B21[b_k,y] += z2*(phi_x+v_2*phi_4*phi_pi)
    B21[b_k,pi] += z3*(phi_x+v_2*phi_4*phi_pi)
    B20[b_k,i] += -z4*(phi_x+v_2*phi_4*phi_pi)
    B21[b_k,q] += z5*(phi_x+v_2*phi_4*phi_pi)
    B20[b_k,s] += -z6*(phi_x+v_2*phi_4*phi_pi)
    B20[b_k,tau] +=-z7*(phi_x+v_2*phi_4*phi_pi)
    B21[b_k,q_k] += z8*(phi_x+v_2*phi_4*phi_pi)
    B20[b_k,b_k] += -z9*(phi_x+v_2*phi_4*phi_pi)
    B23[b_k,e] += z10*(phi_x+v_2*phi_4*phi_pi)
    B23[b_k,r] += z11*(phi_x+v_2*phi_4*phi_pi)
    #B20[b,y] = v_2*phi_2 
    B20[b_k,k] += -z1*(-v_2*phi_2)
    B21[b_k,y] += z2*(-v_2*phi_2)
    B21[b_k,pi] += z3*(-v_2*phi_2)
    B20[b_k,i] += -z4*(-v_2*phi_2)
    B21[b_k,q] += z5*(-v_2*phi_2)
    B20[b_k,s] += -z6*(-v_2*phi_2)
    B20[b_k,tau] +=-z7*(-v_2*phi_2)
    B21[b_k,q_k] += z8*(-v_2*phi_2)
    B20[b_k,b_k] += -z9*(-v_2*phi_2)
    B23[b_k,e] += z10*(-v_2*phi_2)
    B23[b_k,r] += z11*(-v_2*phi_2)
  
    B11 = m(inv(B10),B11)*aa;B12 = m(inv(B10),B12)*aa;B13 = m(inv(B10),B13)*aa;B14 = m(inv(B10),B14)*aa;B21 = m(inv(B20),B21)*aa;B22 = m(inv(B20),B22)*aa;B23 = m(inv(B20),B23)*aa
    
    
    # B10*Y[t] = B11*EY[t+1]+B12*M[t]+B13*U[t]+B14*EM[t+1]
    # M[t]= B21*EY[t+1]+B22*M[t-1]+B23*U[t]
    
    PSI_tilde=PSI[0:5]*aa
    return B11,B12,B13,B14,B21,B22,B23,PSI_tilde

def Set_M_ZLB(para,PSI,aa):
    # Initialize Parameters
    #phi_x = para[12];phi_pi = para[13];rh0= para[15];
    para1 = para+0
    para1[14] = 0; 
    para1[12] = 0
    para1[13] = 0
    B11,B12,B13,B14,B21,B22,B23,PSI_tilde = Set_M(para1,PSI,aa)
    return B11,B12,B13,B14,B21,B22,B23

    
para = set_para()


In [11]:
# Key
m_qk = 0;m_pi = 1
m_q = 2;m_x = 3
m_s = 4;m_b = 5
m_tau = 6;m_i = 7
T = 203
Likic = cp.zeros(T)
data = pd.read_csv(r'/home/aj/Downloads/Research Files/y_raw.csv')
y_raw = cp.array(np.array(data))
Y_data = y_raw.T
Y_data[m_qk] *=1e-1
Y_data[m_pi] *=1e-2
Y_data[m_pi] = Y_data[m_pi] - cp.mean(Y_data[m_pi]) 
Y_data[m_q] *=1e-2
Y_data[m_x] *=1e-3
Y_data[m_s] *=1e3
Y_data[m_b] *=1e-1
Y_data[m_tau] *=1e1
Y_data[m_i] *=1e0
m = cp.matmul
stack = cp.vstack
arr = cp.array
inv = cp.linalg.inv
def Init(para,aa):
    T = 203
    R1,A11_z,A21_z,C11_z,C21_z,PSI1_z,A111_z,A121_z,C111_z,C121_z = Solve_ZLB(para);
    R1,A11,A21,C11,C21,PSI1,A111,A121,C111,C121 = Solve(para);
    B111_z,B121_z,B131_z,B141_z,B211_z,B221_z,B231_z = Set_M_ZLB(para,cp.asnumpy(PSI1),aa)
    B111,B121,B131,B141,B211,B221,B231,PSI1_tilde = Set_M(para,cp.asnumpy(PSI1),aa);    



    R = arr(R1);B11 = arr(B111);B12 = arr(B121);B13 = arr(B131);B14 = arr(B141);
    B21 = arr(B211);B22 = arr(B221);B23 = arr(B231);

    B11_z = arr(B111_z);B12_z = arr(B121_z);B13_z = arr(B131_z);B14_z = arr(B141_z);
    B21_z = arr(B211_z);B22_z = arr(B221_z);B23_z = arr(B231_z);


    G0 = arr(A111)*aa;G1 = arr(A121)*aa;
    Q0 = arr(C111)*aa;Q1 = arr(C121)*aa
    Q0_z = arr(C111_z)*aa;Q1_z = arr(C121_z)*aa;
    G0_z = arr(A111_z)*aa;G1_z = arr(A121_z)*aa;
    PSI = arr(PSI1)
    return Q0,G0,Q1,G1,Q0_z,G0_z,Q1_z,G1_z,PSI,B11,B12,B13,B14,B21,B22,B23,B11_z,B12_z,B13_z,B14_z,B21_z,B22_z,B23_z,R

In [ ]:
x = 0;
pi = 1;
q = 2;
qk = 3;
b = 4;
k = 5;
q_km = 6;
b_km = 7;
w = 8;
tau = 9;
b_k = 10;
i = 11;

r = 12;
a = 13;
s = 14;
u = 15;
e = 16;
aa = 1e-2
####################################### Actual Data #####################################
#data = pd.read_csv(r'/home/aj/Downloads/Research Files/y_raw.csv')
#data = pd.read_csv(r'/home/aj/Downloads/Research Files/USE.csv')
# Using this one:
data = pd.read_csv(r'/home/aj/Downloads/Research Files/USEFFR.csv')

#data = pd.read_csv(r'/home/aj/Documents/DATA.csv')

y_raw = cp.array(data[:]).T
#YY = stack([Y_data[m_x],Y_data[m_pi],Y_data[m_q],Y_data[m_qk],Y_data[m_b],Y_data[m_tau],Y_data[m_s]])
y_raw[0,:] = y_raw[0,:]-cp.mean(y_raw[0,:])
y_raw[1,:] = y_raw[1,:]-cp.mean(y_raw[1,:])
y_raw[m_i]*=4
data.head()

In [ ]:
def plot_full(Y_pred,z1,z2):
    m_pi = 0;m_x = 1;m_s =2;m_tau =3;m_b = 4;m_q = 5;m_qk = 6;m_i = 7 
    x = 0;pi = 1;q = 2;qk = 3;b = 4;k = 5;q_km = 6;b_km = 7;w = 8;tau = 9;b_k = 10;i = 11;r = 12;a = 13;s = 14;u = 15;e = 16;
    T = z2-z1
    fig1, ax1= plt.subplots()
    ax1.plot(np.arange(T), cp.asnumpy(y_raw[m_x,z1:z2]))
    ax1.plot(np.arange(T), cp.asnumpy(Y_pred[m_x,z1:z2]))
    ax1.set_title("Output ")
    ax1.set_xlabel("Time")
    fig2, ax2= plt.subplots()
    ax2.plot(np.arange(T), cp.asnumpy(y_raw[m_pi,z1:z2]))
    ax2.plot(np.arange(T), cp.asnumpy(Y_pred[m_pi,z1:z2]))
    ax2.set_title("Inflation ")
    ax2.set_xlabel("Time")
    fig3, ax3= plt.subplots()
    ax3.plot(np.arange(T), cp.asnumpy(y_raw[m_i,z1:z2]))
    ax3.plot(np.arange(T), cp.asnumpy(Y_pred[m_i,z1:z2]))
    ax3.set_title("FFR")
    ax3.set_xlabel("Time")
    fig4, ax4= plt.subplots()
    ax4.plot(np.arange(T), cp.asnumpy(y_raw[m_q,z1:z2]))
    ax4.plot(np.arange(T), cp.asnumpy(Y_pred[m_q,z1:z2]))
    ax4.set_title("T-Bill Return")
    ax4.set_xlabel("Time")
    fig5, ax5= plt.subplots()
    ax5.plot(np.arange(T), cp.asnumpy(y_raw[m_qk,z1:z2]))
    ax5.plot(np.arange(T), cp.asnumpy(Y_pred[m_qk,z1:z2]))
    ax5.set_title("Corp Bond Return")
    ax5.set_xlabel("Time")
    fig7, ax7= plt.subplots()
    ax7.plot(np.arange(T), cp.asnumpy(y_raw[m_s,z1:z2]))
    ax7.plot(np.arange(T), cp.asnumpy(Y_pred[m_s,z1:z2]))
    ax7.set_title("Seign")
    ax7.set_xlabel("Time")
    fig8, ax8= plt.subplots()
    ax8.plot(np.arange(T), cp.asnumpy(y_raw[m_tau,z1:z2]))
    ax8.plot(np.arange(T), cp.asnumpy(Y_pred[m_tau,z1:z2]))
    ax8.set_title("Taxes")
    ax8.set_xlabel("Time")
    fig9, ax9= plt.subplots()
    ax9.plot(np.arange(T), cp.asnumpy(y_raw[m_b,z1:z2]))
    ax9.plot(np.arange(T), cp.asnumpy(Y_pred[m_b,z1:z2]))
    ax9.set_title("Debt")
    ax9.set_xlabel("Time")

In [ ]:

def ab(a,b,loops):
    for i in range(loops):
        c = cp.matmul(a,b,out=None)
    return c
def prop(para,y_raw):
    loops = 1
    # Capital Share & Depreciation
    para[2] = .3;
    para[3] = .025;
    # Duration of Bonds
    para[4] = .9059;
    para[5] = .89;
    # Proportion of taxes to Capital
    para[16] = .5
    Q0,G0,Q1,G1,Q0_z,G0_z,Q1_z,G1_z,PSI,B11,B12,B13,B14,B21,B22,B23,B11_z,B12_z,B13_z,B14_z,B21_z,B22_z,B23_z,R =Init(para,aa)    
    m_pi = 0;m_x = 1;m_s =2;m_tau =3;m_b = 4;m_q = 5;m_qk = 6;m_i = 7 
    y = 0;pi = 1;q = 2;q_k = 3;
    b = 0;k = 1;q_km = 2;b_km = 3;w = 4;tau = 5;b_k = 6;i = 7;
    r = 0;a = 1;s = 2;u = 3;e = 4
    PSI*=1e-2;J=5000
    M_bar = cp.zeros([8,17]);
    M_bar[m_pi,pi] = 1e0;
    M_bar[m_x,x] = 1e0;
    M_bar[m_s,s] = 1e0; 
    M_bar[m_tau,tau] = 1e0;
    M_bar[m_b,b] = 1e0;
    M_bar[m_q,q] = 1e0;
    M_bar[m_qk,qk] = 1e0;
    LL = cp.zeros([T])
    Y = 1e-3*abs(norm([J,8,1]))
    M = 1e-2*abs(norm([J,8,1]))
    EM =1e-2*(norm([J,8,1])) 
    EM1 =1e-2*norm([J,8,1])
    EY = 1e0*norm([J,4,1])
    EY1 = 1e0*cp.zeros([J,8,1])
    Q_0 = cp.zeros([J,8,5])+Q0
    Q_1 = cp.zeros([J,8,8])+Q1
    G_0 = cp.zeros([J,4,5])+G0
    G_1 = cp.zeros([J,4,8])+G1
    Q_0_z = cp.zeros([J,8,5])+Q0_z
    Q_1_z = cp.zeros([J,8,8])+Q1_z
    G_0_z = cp.zeros([J,4,5])+G0_z
    G_1_z = cp.zeros([J,4,8])+G1_z
    PHI = cp.hstack([resh(Q_0,[J,-1,1]),resh(Q_1,[J,-1,1]),resh(G_0,[J,-1,1]),resh(G_1,[J,-1,1])])
    PHI_z = cp.hstack([resh(Q_0_z,[J,-1,1]),resh(Q_1_z,[J,-1,1]),resh(G_0_z,[J,-1,1]),resh(G_1_z,[J,-1,1])])
    Y_pred = cp.zeros([8,T]);
    Z_lag = 1e-2*norm([J,17,1]);Y_FOR= cp.zeros([8,T]);PROB = cp.zeros([T])
    g =para[29];p=1;s=1e-1
    for t in range(T):
        cp.random.seed(t)
        eps =cp.random.multivariate_normal(cp.zeros([5]),PSI[0:5,:],J);
        r = resh(cp.diag(R),[1,5,1])
        if t==0:
            r_star = para[28]*cp.random.standard_normal([J])
            U = resh(eps,[J,5,1])
            U[:,0,:]+=para[22]*resh(r_star,[-1,1])
        else:
            U = resh(U,[J,5,1])*r+resh(eps,[J,5,1])
            r_star = r_star + para[28]*cp.random.standard_normal([J])
            U[:,0]+=para[22]*resh(r_star,[-1,1])
        if p==1:
            EM   = (ab(Q_1,M,loops)+ab(Q_0,U,loops))
            U_r  = (U*r)
            EM1  = (ab(Q_0,U_r,loops)+ab(Q_1,EM,loops))
            EY  = (ab(G_0,U,loops)+ab(G_1,EM,loops))
            EY1  = (ab(G_0,U_r,loops)+ab(G_1,EM1,loops))
            M    = ab(B21,EY1,loops) +ab(B22,M,loops)+ab(B23,U,loops)
        else:
            EM   = (ab(Q1,M,loops)+ab(Q0,U,loops))
            U_r  = (U*r)
            EM1  = (ab(Q0,U_r,loops)+ab(Q1,EM,loops))
            EY  = (ab(G0,U,loops)+ab(G1,EM,loops))
            EY1  = (ab(G0,U_r,loops)+ab(G1,EM1,loops))
            M    = ab(B21,EY1,loops) +ab(B22,M,loops)+ab(B23,U,loops)
        M[cp.where(M[:,i]<=0)[0],i] = 0 
        I = M[:,i]
        Y    = ab(B11,EY1,loops) +ab(B12,M,loops)+ab(B12,M,loops)+ab(B13,U,loops)+ab(B14,EM1,loops)
        if p==1:
            X = cp.zeros([J,12,156])
            X[:,0:8,0:40] = cp.kron(cp.eye(8),resh(U,[J,1,5]))
            X[:,0:8,40:104] = cp.kron(cp.eye(8),resh(M,[J,1,8]))
            X[:,8:,104:136] = cp.kron(cp.eye(4),resh(EM,[J,1,8]))
            X[:,8:,136:] = cp.kron(cp.eye(4),resh(U,[J,1,5]))
            ERR_y  = Y - EY;ERR_m  = M - EM
            ERR = cp.hstack([ERR_m,ERR_y])
            PHI = PHI + g*ab(cp.swapaxes(X,2,1),ERR,1)
            PHI_z = PHI_z + g*ab(cp.swapaxes(X,2,1),ERR,1)
            prob = .1*cp.array(sc.stats.norm.cdf(0,cp.asnumpy(EM[:,-1,:]),1))
            PROB[t] = cp.mean(prob)*1
            K =resh(cp.tile(resh(prob,[-1,1]),[8*8]),[-1,8,8])
            Q_0 = (1-K[-1,0:8,0:5])*resh(PHI[:,0:40],[-1,8,5]) +K[-1,0:8,0:5]*resh(PHI_z[:,0:40],[-1,8,5])
            Q_1 = (1-K[-1,0:8,0:8])*resh(PHI[:,40:104],[-1,8,8]) +K[-1,0:8,0:8]*resh(PHI_z[:,40:104],[-1,8,8])
            G_0 = (1-K[-1,0:4,0:5])*resh(PHI[:,104:124],[-1,4,5])+K[-1,0:4,0:5]*resh(PHI_z[:,104:124],[-1,4,5])
            G_1 = (1-K[-1,0:4,0:8])*resh(PHI[:,124:],[-1,4,8])+K[-1,0:4,0:8]*resh(PHI_z[:,124:],[-1,4,8])

        Z = cp.hstack([Y,M,U])
        YY = ab(M_bar,Z,loops)- ab(M_bar,Z_lag,loops)
        YY[:,m_i] = I*4e1
        YY[:,m_x] *= 1e2
        YY[:,m_pi] *= 1e1
        YY[:,m_q] *= 4e2
        YY[:,m_qk] *= 4e2
        YY[:,m_s] *= 4e1
        YY[:,m_tau] *= 4e2
        YY[:,m_b] *= 4e2
        if t==0:
            Y_FOR[:,t] = resh(cp.mean(YY*s,0),[-1])
            err = (YY*s-resh(y_raw[:,t],[1,-1,1]))
        else:
            Y_FOR[:,t] = resh(cp.sum(resh(w,[-1,1,1])*YY*s,0),[-1])
            err = (resh(w,[-1,1,1])*YY*s-resh(y_raw[:,t],[1,-1,1]))
        c1 = (2*math.pi**(-4))*((1e-16)**-.5)
        err = (YY*s-resh(y_raw[:,t],[1,-1,1]))**2
        w = c1*cp.exp(-0.5*(cp.sum(err*1e2,1)))

        LL[t] =-cp.log(J)+ cp.log(cp.sum(w))
        #w = 10*cp.exp(-1e2*(cp.sum(err,1)))
        w = cp.asnumpy(w/cp.sum(w))
        
        try:
            idx = np.random.choice(np.arange(J), size=int(J), replace=True, p=resh(w,[-1]))
        except:
            idx = cp.arange(J)
        if p==1:
            Q_0 = Q_0[idx]
            G_0 = G_0[idx]
            Q_1 = Q_1[idx]
            G_1 = G_1[idx]
        M = M[idx]        
        U = U[idx]
        Y = Y[idx]
        r_star = r_star[idx]
        w = cp.array(w)
        Y_pred[:,t] = resh(cp.sum(resh(w,[-1,1,1])*YY*s,0),[-1]) 
        Z_lag = Z[idx]
    SSE = cp.sum((Y_FOR*s-y_raw)**2)
    likic = cp.sum(LL)
    return likic,SSE

In [ ]:
def param_checks(para):
    go_on = 0
    # check params are in bounds
    if np.any(para<0)==True:
        go_on = 0
        return go_on
    #if  para[16]>=1: 
    #    go_on = 0
    #    return go_on
    if np.any(para[17:23]>=1):
        go_on = 0
        return go_on
    #if para[6]<1:
    #    go_on = 0
    #    return go_on
    if para[15]>=1:
        go_on = 0
        return go_on
    # Check Determinacy Conditions
    try:
        det,det_zlb = check_det(para)
    except:
        go_on = 0
        return go_on
    if det !=4:
        go_on =0
        return go_on
    else:
        go_on = 1
    return go_on


In [ ]:
def check_det(para):
    m = np.matmul
    inv = np.linalg.inv
    # Define Variables
    psi = para[0];epsilon = para[1];alpha = para[2];sigma = para[3];rho = para[4];
    rho_k = para[5];delta = para[6];n1 = para[7];n2 = para[7];phi_b = para[9];
    phi_s= para[10];theta = para[11];phi_x = para[12];phi_pi = para[13];phi_t =para[14];rh0= para[15];h_w = para[16];h_r = 1-h_w;p_u = para[17];p_r = para[18];p_s = para[19];p_e = para[20];p_a = para[21];p_r_star = para[22];
    p_r_star = para[22];sig_u =para[23];sig_r = para[24];sig_s = para[25];
    sig_e = para[26];sig_a = para[27];sig_r_star = para[28]
    h_r = 1-h_w;
    r = 0;a = 1;s = 2;u = 3;e = 4;
    r_sh = 0;a_sh = 1;s_sh = 2;u_sh = 3;e_sh = 4;i_sh = 5
    b = 5;k = 6;q_km = 7;b_km = 8;w = 9;tau = 10;b_k = 11;i = 12
    y = 13;pi = 14;q = 15;q_k = 16
    C_bar = .6;K_bar = .4;Y_bar = 1;c1 = Y_bar/(Y_bar-K_bar);c2 = K_bar/(Y_bar-K_bar)
    # Set up nuisance parameters
    alpha_tilde = (1-(1-.99*theta)*(1-theta))**-1;
    gam = alpha*(psi**-1)*(-epsilon);
    v_1 = alpha_tilde*.99*theta;
    v_2 = alpha_tilde*((1-.99*theta)/(1-gam))
    phi_1 = (1-alpha*(psi**-1));phi_2 = alpha*(psi**-1)*delta*c1;phi_3 = alpha+(psi**-1)*alpha*delta*c2
    phi_4 = alpha*(psi**-1)*h_w;Q = (C_bar**delta)/((sigma*K_bar)**(n2))
    # Generate Matrices
    GAM0 = np.zeros([17,17]);GAM1 = np.zeros([17,17]);PSI = np.zeros([17,6])    
    # EQ 0 Goverment Bond Demand
    GAM0[b,b] =1
    GAM1[b,b] =1
    GAM0[b,q] = (1-rho)
    GAM0[b,pi] =1
    GAM0[b,tau] = 1
    GAM0[b,s] = 1
    # EQ 1 Capital Evolution 
    GAM0[k,k] =1
    GAM1[k,k] =1-sigma
    GAM0[k,b_km] =-1*( 1-2*sigma)
    GAM0[k,q_km] =-1*(1-2*sigma)
    GAM1[k,b_km] = -2*sigma
    GAM1[k,q_km] = -2*sigma
    GAM1[k,e] = -2*sigma
    # EQ 2 tech shock
    GAM0[a,a] = 1
    GAM1[a,a] = p_a
    PSI[a,a_sh] = sig_a
    # EQ 3 Seinorage shock
    GAM0[s,s] = 1 
    GAM1[s,s] = p_s
    PSI[s,s_sh] = sig_s
    # EQ 4 Demand shock
    GAM0[r,r] = 1
    GAM0[r,i] = p_r_star
    GAM1[r,r] = p_r 
    PSI[r,r_sh] = sig_r 
    # EQ 5 Cost Push shock
    GAM0[u,u] = 1 
    GAM1[u,u] = p_u 
    PSI[u,u_sh] = sig_u 
    # EQ 7 Investment "Credit Freeze" shock
    GAM0[e,e] = 1 
    GAM1[e,e] = p_e 
    PSI[e,e_sh] = sig_e 
    # EQ 8 coporate bond price lag
    GAM0[q_km,q_km] = 1
    GAM1[q_km,q_k] = 1
    # EQ 9 coporate bond demand lag
    GAM0[b_km,b_km] = 1
    GAM1[b_km,b_k] = 1
    # EQ 10 Money/IS equation
    GAM1[y,y] = c1
    GAM1[y,k] = c2
    GAM0[y,y] = c1*delta**-1
    GAM0[y,k] = -c2*delta**-1
    GAM0[y,pi] = delta**-1
    GAM0[y,tau] = -h_r*delta**-1
    #GAM1[y,i] = delta**-1
    GAM1[y,r] = -delta**-1
    # EQ 11 Phillips Curve
    GAM1[pi,pi] = 1
    GAM0[pi,pi] =v_1
    GAM1[pi,w] = -v_2*phi_1
    GAM1[pi,y] = -v_2*phi_2
    GAM1[pi,k] = v_2*phi_3
    GAM0[pi,tau] = v_2*phi_4
    GAM0[pi,u] =1
    # EQ 12 Gov Bond Pricing
    GAM1[q,q] = n1
    GAM1[q,b] = n1
    GAM1[q,y] = -delta*c1
    GAM1[q,k] = delta*c2
    GAM0[q,y] = .99*delta*c1
    GAM0[q,k] = -.99*delta*c2
    GAM0[q,pi] = .99*delta
    GAM0[q,tau] = h_r
    GAM0[q,q] = -.99*rho
    # EQ 13 Corp Bond Pricing
    GAM1[q_k,q] = 1-n2-Q**-1
    GAM1[q_k,b] = n2
    GAM1[q_k,y] = -delta*c1*(Q**-1)
    GAM1[q_k,k] = delta*c2*(Q**-1)
    GAM0[q_k,y] = .99*delta*c1*.994*(Q**-1)
    GAM0[q_k,k] = -.99*delta*c2*.994*(Q**-1)
    GAM0[q_k,pi] = .99*delta*.994*(Q**-1)
    GAM0[q_k,tau] = h_r*.994*(Q**-1)
    GAM0[q_k,q_k] = -(.99*rho_k*.994-.99*rho_k*.42*.006)*(Q**-1)
    # EQ 14 Monetary Policy
    GAM0[i,i] = 1
    GAM1[i,i] = 1
    PSI[i,i_sh] =sig_r_star
    # EQ 15 Wages
    GAM0[w,w] = 1-alpha*psi
    GAM0[w,q_k] = -2*alpha*psi
    GAM0[w,b_k] = -2*alpha*psi
    GAM1[w,q_k] = -2*alpha*psi
    GAM1[w,b_k] = -2*alpha*psi
    GAM0[w,e] = -2*alpha*psi
    GAM0[w,y] = -(delta-psi)*c1
    GAM0[w,a] = -psi
    GAM0[w,k] = delta*c2 
    GAM0[w,tau] = h_w 
    # EQ 16 Tax LOM
    GAM0[tau,tau] =1    
    GAM0[tau,b] = -phi_b
    GAM0[tau,q] = -phi_b 
    GAM0[tau,s] = phi_s 
    GAM1[tau,tau] = phi_t 

    # EQ 17 Corporate Bond Demand
    GAM0[b_k,b_k] = 1
    GAM0[b_k,w] = -1
    GAM1[b_k,q_k] = 1
    GAM1[b_k,b_k] = 1
    GAM0[b_k,y] = -(1-alpha)**-1
    GAM0[b_k,a] = (1-alpha)**-1
    GAM0[b_k,k] = (1-alpha)**-1
    GAM0[b_k,e] =  1
    GAM0[b_k,q_k] = 1

    n_s = 13;n_j = 4
    VV = np.zeros(17)
    F = m(inv(GAM0),GAM1);e_vals, e_vecs = eig(F)
    for i in range(17):
        VV[i] =  ((np.real(e_vals[i])**2)+(np.imag(e_vals[i])**2))**.5
    # Define Variables
    psi = para[0];epsilon = para[1];alpha = para[2];sigma = para[3];rho = para[4];rho_k = para[5];delta = para[6];n1 = para[7];n2 = para[7];phi_b = para[9];phi_s= para[10];theta = para[11];phi_x = para[12];phi_pi = para[13];phi_t =para[14];rh0= para[15];h_w = para[16];h_r = 1-h_w;p_u = para[17];p_r = para[18];p_s = para[19];p_e = para[20];p_a = para[21];p_r_star = para[22];
    p_r_star = para[22];sig_u =para[23];sig_r = para[24];sig_s = para[25];sig_e = para[26];sig_a = para[27];sig_r_star = para[28]
    h_r = 1-h_w;
    r = 0;a = 1;s = 2;u = 3;e = 4;
    r_sh = 0;a_sh = 1;s_sh = 2;u_sh = 3;e_sh = 4;i_sh = 5
    b = 5;k = 6;q_km = 7;b_km = 8;w = 9;tau = 10;b_k = 11;i = 12
    y = 13;pi = 14;q = 15;q_k = 16
    C_bar = .6;K_bar = .4;Y_bar = 1;c1 = Y_bar/(Y_bar-K_bar);c2 = K_bar/(Y_bar-K_bar)
    # Set up nuisance parameters
    alpha_tilde = (1-(1-.99*theta)*(1-theta))**-1;gam = alpha*(psi**-1)*(-epsilon);v_1 = alpha_tilde*.99*theta;v_2 = alpha_tilde*((1-.99*theta)/(1-gam))
    phi_1 = (1-alpha*(psi**-1));phi_2 = alpha*(psi**-1)*delta*c1;phi_3 = alpha+(psi**-1)*alpha*delta*c2
    phi_4 = alpha*(psi**-1)*h_w;Q = (C_bar**delta)/((sigma*K_bar)**(n2))
    # Generate Matrices
    GAM0 = np.zeros([17,17]);GAM1 = np.zeros([17,17]);PSI = np.zeros([17,6])    
    # EQ 0 Goverment Bond Demand
    GAM0[b,b] =1
    GAM1[b,b] =1
    GAM0[b,q] = (1-rho)
    GAM0[b,pi] =1
    GAM0[b,tau] = 1
    GAM0[b,s] = 1
    # EQ 1 Capital Evolution 
    GAM0[k,k] =1
    GAM1[k,k] =1-sigma
    GAM0[k,b_km] =-1*( 1-2*sigma)
    GAM0[k,q_km] =-1*(1-2*sigma)
    GAM1[k,b_km] = -2*sigma
    GAM1[k,q_km] = -2*sigma
    GAM1[k,e] = -2*sigma
    # EQ 2 tech shock
    GAM0[a,a] = 1
    GAM1[a,a] = p_a
    PSI[a,a_sh] = sig_a
    # EQ 3 Seinorage shock
    GAM0[s,s] = 1 
    GAM1[s,s] = p_s
    PSI[s,s_sh] = sig_s
    # EQ 4 Demand shock
    GAM0[r,r] = 1
    GAM0[r,i] = p_r_star
    GAM1[r,r] = p_r 
    PSI[r,r_sh] = sig_r 
    # EQ 5 Cost Push shock
    GAM0[u,u] = 1 
    GAM1[u,u] = p_u 
    PSI[u,u_sh] = sig_u 
    # EQ 7 Investment "Credit Freeze" shock
    GAM0[e,e] = 1 
    GAM1[e,e] = p_e 
    PSI[e,e_sh] = sig_e 
    # EQ 8 coporate bond price lag
    GAM0[q_km,q_km] = 1
    GAM1[q_km,q_k] = 1
    # EQ 9 coporate bond demand lag
    GAM0[b_km,b_km] = 1
    GAM1[b_km,b_k] = 1
    # EQ 10 Money/IS equation
    GAM1[y,y] = c1
    GAM1[y,k] = c2
    GAM0[y,y] = c1*delta**-1
    GAM0[y,k] = -c2*delta**-1
    GAM0[y,pi] = delta**-1
    GAM0[y,tau] = -h_r*delta**-1
    #GAM1[y,i] = delta**-1
    GAM1[y,r] = -delta**-1
    # EQ 11 Phillips Curve
    GAM1[pi,pi] = 1
    GAM0[pi,pi] =v_1
    GAM1[pi,w] = -v_2*phi_1
    GAM1[pi,y] = -v_2*phi_2
    GAM1[pi,k] = v_2*phi_3
    GAM0[pi,tau] = v_2*phi_4
    GAM0[pi,u] =1
    # EQ 12 Gov Bond Pricing
    GAM1[q,q] = n1
    GAM1[q,b] = n1
    GAM1[q,y] = -delta*c1
    GAM1[q,k] = delta*c2
    GAM0[q,y] = .99*delta*c1
    GAM0[q,k] = -.99*delta*c2
    GAM0[q,pi] = .99*delta
    GAM0[q,tau] = h_r
    GAM0[q,q] = -.99*rho
    # EQ 13 Corp Bond Pricing
    GAM1[q_k,q] = 1-n2-Q**-1
    GAM1[q_k,b] = n2
    GAM1[q_k,y] = -delta*c1*(Q**-1)
    GAM1[q_k,k] = delta*c2*(Q**-1)
    GAM0[q_k,y] = .99*delta*c1*.994*(Q**-1)
    GAM0[q_k,k] = -.99*delta*c2*.994*(Q**-1)
    GAM0[q_k,pi] = .99*delta*.994*(Q**-1)
    GAM0[q_k,tau] = h_r*.994*(Q**-1)
    GAM0[q_k,q_k] = -(.99*rho_k*.994-.99*rho_k*.42*.006)*(Q**-1)
    # EQ 14 Monetary Policy
    GAM0[i,i] = 1
    GAM1[i,i] = 1
    PSI[i,i_sh] =sig_r_star
    # EQ 15 Wages
    GAM0[w,w] = 1-alpha*psi
    GAM0[w,q_k] = -2*alpha*psi
    GAM0[w,b_k] = -2*alpha*psi
    GAM1[w,q_k] = -2*alpha*psi
    GAM1[w,b_k] = -2*alpha*psi
    GAM0[w,e] = -2*alpha*psi
    GAM0[w,y] = -(delta-psi)*c1
    GAM0[w,a] = -psi
    GAM0[w,k] = delta*c2 
    GAM0[w,tau] = h_w 
    # EQ 16 Tax LOM
    GAM0[tau,tau] =1    
    GAM0[tau,b] = -phi_b
    GAM0[tau,q] = -phi_b 
    GAM0[tau,s] = phi_s 
    GAM1[tau,tau] = phi_t 

    # EQ 17 Corporate Bond Demand
    GAM0[b_k,b_k] = 1
    GAM0[b_k,w] = -1
    GAM1[b_k,q_k] = 1
    GAM1[b_k,b_k] = 1
    GAM0[b_k,y] = -(1-alpha)**-1
    GAM0[b_k,a] = (1-alpha)**-1
    GAM0[b_k,k] = (1-alpha)**-1
    GAM0[b_k,e] =  1
    GAM0[b_k,q_k] = 1

    n_s = 13;n_j = 4
    VV_zlb = np.zeros(17)
    F = m(inv(GAM0),GAM1);
    e_vals, e_vecs = eig(F)
    for i in range(17):
        VV_zlb[i] =  ((np.real(e_vals[i])**2)+(np.imag(e_vals[i])**2))**.5
    idx = np.argsort(VV_zlb)
    det_zlb = np.sum(VV_zlb>=1)
    det = np.sum(VV>=1)

    return det,det_zlb
def NK_priors(para):
    P = np.zeros([30])
    #Frisch
    P[0] = sc.stats.norm.logpdf(para[0],loc=3**-1,scale=1**2)
    # Markup
    mu = 1.2;sig = .1**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[1] = sc.stats.gamma.logpdf(para[1],a=k,scale=thetaa)
    # Capital Share
    #mu = .3;sig = .01**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    #P[2] = sc.stats.gamma.logpdf(para[2],a=k,scale=thetaa)
    # Depreciation
    #mu = .15;sig = .01**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    #P[3] = sc.stats.gamma.logpdf(para[3],a=k,scale=thetaa)
    #CRRA 
    mu = 3.1;sig = 1**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[6] = sc.stats.gamma.logpdf(para[6],a=k,scale=thetaa)    
    #Wealth in utility
    #P[7] = sc.stats.norm.logpdf(para[7],loc=1.1**-1,scale=.5**2)
    #P[8] = 0
    #Calvo
    mu = .1;sig = .15**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[11] = sc.stats.gamma.logpdf(para[11],a=k,scale=thetaa)
    #Taylor Coeff
    mu = 0.5;sig = .15**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[12] = sc.stats.gamma.logpdf(para[12],a=k,scale=thetaa)    
    mu = 1.5;sig = .15**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[13] = sc.stats.gamma.logpdf(para[13],a=k,scale=thetaa)
    mu = .7;sig = .35**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[15] = sc.stats.gamma.logpdf(para[15],a=k,scale=thetaa)
    #Tax Persistance
    mu = .7;sig = .1**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[14] = sc.stats.gamma.logpdf(para[14],a=k,scale=thetaa)
    # Tax to Debt
    P[9] = sc.stats.norm.logpdf(para[9],loc=0.2,scale=.05**2)
    #Variances
    mu = .5;sig = 1**2;alphaa = (mu**2/sig)+2;betaa = mu*(alphaa-1)    
    P[23:29] = sc.stats.invgamma.logpdf(para[23:29],a=alphaa,scale=betaa)
    P[28] = 0;
    #RLS
    mu = .031;sig = .022**2;thetaa = (mu**-1)*sig;k = mu*(thetaa**-1)
    P[29] = sc.stats.gamma.logpdf(para[29],a=k,scale=thetaa)
    return np.sum(P)

In [ ]:
c1 = 0.02
#para[0] =psi;
para[0] = 3.6**-1
#para[1] = epsilon;
para[1] = 1.2
#para[2] = alpha;
para[2] = .3
#para[3] = sigma;
para[3] = .025
#para[4] = rho;
para[4] =.9059
#para[5] = rho_k;
para[5] = .89
#para[6] = delta;
para[6] = .33**-1
#para[7] = n1;
para[7] =.7
#para[8] = n2;
para[8] =.7
#para[9]= phi_b;
para[9]= .2
#para[10] =phi_s;
para[10]= .2
#para[11] = theta;
para[11]= .3
#para[12] = phi_x;
para[12]= .5
#para[13] = phi_pi;  
para[13]= 2.0
#para[14] = phi_t;
para[14]= 0.7;
#para[15] = rh0;  
para[15]= .7
#para[16] = h_w;
para[16]= .5
#para[17] = p_u;
para[17]= .5
#para[18] = p_r;
para[18]= .5
#para[19] = p_s;
para[19]= .5
#para[20] = p_e;
para[20]= .5
#para[21] = p_a;
para[21]= .5
#para[22] = p_r_star;
para[22]= 0.5
#para[23] = sig_u
para[23] = c1
#para[24] = sig_r
para[24] = c1
#para[25] = sig_s
para[25] = c1
#para[26] = sig_e
para[26] = c1
#para[27] = sig_a
para[27] = c1
#para[28] = sig_r_star
para[28] = c1
#para[29] = g
para[29] = .03

In [ ]:
# Start the MCMC
# Note: The Frisch you set is 1.6**-1, para[0] = Frisch**-1 
m = 800
Nsim = 100000+5000
npara = 30
go_on = 0
Thetasim = np.zeros([Nsim,npara])
likely   = np.zeros([Nsim,npara])
logpost = np.zeros(Nsim)
ERRS = np.zeros(Nsim)
AA = np.zeros(Nsim)

c1 = .02
para[23] = c1;para[24] = c1;para[25] = c1;para[26] = c1;para[27] = c1

# Duration of Bonds
para[4] = .9059;para[5] = .89;
para[29] = .025
Thetasim[0,:] = para
likij,SSE_j =prop(cp.asnumpy(Thetasim[0,:]),y_raw)
obj = likij+NK_priors(cp.asnumpy(Thetasim[0,:]))
logpost[0] =obj
likely[0] =cp.asnumpy(likij)
ERRS[0] = SSE_j 
accept = 0
#P3 = np.cov(np.load('/home/aj/Downloads/Research Files/FTTPL/PostDIST.npy')[1000:85000],rowvar = False)
P3 = np.eye(npara)
print('log posterior:',obj)
print('log likelihood:',likij)
print('SSE:',SSE_j)
print('Determinate:',param_checks(para)==1)
AR = cp.zeros([m+1]);v = 0;MA = 0;i = 0;c = 1e-4;go_on = 0

In [ ]:
adapt = 1;random_block = 0;
if random_block ==1:
    CO = np.zeros([30,30])
    CO[0:5,0:5] =1;CO[5:10,5:10] =1;CO[10:10,10:15] =1;CO[15:20,15:20] =1;CO[20:25,20:25] =1
    CO[25:,25:] =1;
while i<Nsim:
    #print(i)
    if adapt ==1:
        if (x % 10000 == 1) and x>1: 
            P3 = np.cov(Thetasim[0:i,:],rowvar=False)
    if adapt==0:
        if i == 10000:
            P3 = np.cov(Thetasim[0:i,:],rowvar=False)    
    while go_on == 0:
        if random_block ==1:
                rb = np.random.uniform(0,1,30)
                idx = np.argsort(rb)
                Omega = np.zeros([30,30]);
                Omega = P3[idx]
                Omega = Omega[:,idx]*CO
                Thetac = np.zeros(30)
                Thetac[idx] =  np.random.multivariate_normal(Thetasim[i,idx],(c)*np.eye(30)) 
                go_on = param_checks(cp.asnumpy(Thetac))
        if random_block ==0:
            Thetac = np.random.multivariate_normal(Thetasim[i,:],(c)*P3)
            go_on = param_checks(cp.asnumpy(Thetac))
    try:
        likic,SSE_c =prop(cp.asnumpy(Thetac),y_raw)
    except:
        likic = -math.inf
    if cp.isnan(likic):
        likic = -math.inf
    objc = cp.float(NK_priors(cp.asnumpy(Thetac))) + float(likic)
    cp.random.seed(i)
    u = cp.log(cp.random.uniform(0,1))
    #print(u)
    alpha = objc-obj
    mv = alpha - u
    if  mv>=0:
        Thetasim[i+1,:] = Thetac
        likely[i+1,:] = cp.asnumpy(likic) 
        accept = accept+1;
        logpost[i+1] = objc
        ERRS[i+1] = SSE_c
        obj = objc
        #print('accepted')
    else:
        Thetasim[i+1,:] = Thetasim[i,:]
        likely[i+1,:] = cp.asnumpy(likely[i,:]) 
        logpost[i+1] = obj
        ERRS[i+1] = SSE_j
        #print('rejected')
    #print('log_posterior:',obj)
    AR = (accept/(i+1))*100

    if v == m:
                    
                    v = 0
                    
                    if  AR>90:
                                        c = c*2
                    if 90>AR and AR>75:
                                        c = c*1.6
                    if 75>AR and AR>50:
                                        c = c*1.5
                    if 50>AR and AR>45:
                                        c = c*1.3
                    if 45>AR and AR>40:
                                        c = c*1.26
                    if 40>AR and AR>35:
                                        c = c*1.25
                    if 35>AR and AR>33:
                                        c = c*1.05
                    if 33>AR and AR >30:
                                        c = c*1
                    if 20>AR and AR >15:
                                        c = c*.45
                    if 15>AR and AR>10:
                                        c = c*.03
                    if 10>AR and AR >5:
                                        c = c*.01
                    if 5>AR:
                            c = c*.001
    if c > 10:
            c = 1
    AA[i] = AR;
    
    if (i % 5000) == 0:
        print("Iteration: ", i)
        print("Log Post mean: ", cp.mean(logpost[0:i]))
        print("Acceptance Rate: ",AR)
        print("c: ",c)
        print('Frisch:', np.mean(Thetasim[0:i,0],0) ) 
        print('Markup:', np.mean(Thetasim[0:i,1],0) )     
        #print('cobb:', np.mean(Thetasim[0:i,2],0) )
        #print('depreciation:', np.mean(Thetasim[0:i,3],0) )
        print('CRRA:', np.mean(Thetasim[0:i,6],0))
        #print('utility of gov :', np.mean(Thetasim[0:i,7],0) )
        #print('utility of corp:', np.mean(Thetasim[0:i,8],0) )
        print('utility of wealth:', np.mean(Thetasim[0:i,7],0) )
        print('taxes to fiscal:', np.mean(Thetasim[0:i,9],0) )
        print('taxes to siegnorage:', np.mean(Thetasim[0:i,10],0) )
        print('taxes persistance:', np.mean(Thetasim[0:i,14],0))
        print('Calvo:', np.mean(Thetasim[0:i,11],0) )
        print('Taylor X:', np.mean(Thetasim[0:i,12],0))
        print('Taylor Pi:', np.mean(Thetasim[0:i,13],0))
        print('MP peristance:', np.mean(Thetasim[0:i,15],0))
        #print('labor income tax:', np.mean(Thetasim[0:i,16],0))
        print('p_u:', np.mean(Thetasim[0:i,17],0))
        print('p_r:', np.mean(Thetasim[0:i,18],0) )
        print('p_s:', np.mean(Thetasim[0:i,19],0) )
        print('p_e:', np.mean(Thetasim[0:i,20],0) )
        print('p_a:', np.mean(Thetasim[0:i,21],0) )
        #print('p_r_star:', np.mean(Thetasim[0:i,22],0) )
        print('sig_u:', np.mean(Thetasim[0:i,23],0) )
        print('sig_r:', np.mean(Thetasim[0:i,24],0) )
        print('sig_s:', np.mean(Thetasim[0:i,25],0) )
        print('sig_e:', np.mean(Thetasim[0:i,26],0) )
        print('sig_a:', np.mean(Thetasim[0:i,27],0) )
       #print('sig_r_star:', np.mean(Thetasim[0:i,28],0) )
        print('RLS gain:', np.mean(Thetasim[0:i,29],0))
        cp.save('/home/aj/Downloads/Research Files/FTTPL/PostDIST_PF_1', Thetasim)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/Acceptance_PF_1', AA)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/logpost_PF_1', logpost)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/likely_PF_1', likely)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/covmult_PF_1', c)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/iter_PF_1', i)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/ERRS_PF_1', ERRS)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/Covariance_PF_1', P3)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/Accept_PF_1',accept)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/AA_PF_1',AA)
        cp.save('/home/aj/Downloads/Research Files/FTTPL/v_PF_1',v)

    go_on = 0
    i = i+1
    v = v+1